In [1]:
import requests
from lxml import html
import pandas as pd

https://www.gamereplays.org/starcraft2/replays.php?game=33&show=master_replays&sort_by=latest_upload&search=&st=0

In [3]:
base_url = 'https://www.gamereplays.org/starcraft2/replays.php?game=33&show=master_replays&sort_by=latest_upload&search=&st='

indexes = list(range(0,16400,30))

def getHTML(index = 0):
    url = base_url + str(index)
    rawHTML = requests.get(url).content
    return str(rawHTML)

In [4]:
"""for i in indexes:
    with open('../../../../Desktop/replayhtmls.text', "a") as myfile:
            myfile.write(getHTML(index=i))"""

"""with open('../../../../Desktop/replayhtmls.text', "r") as myfile:
        m_html = myfile.read()
        tree = html.fromstring(m_html)"""

In [8]:
games = tree.xpath('//a[@class = "replay_index_button index_download"]/@href')

In [11]:
series = pd.Series(games)

In [19]:
series.to_clipboard()

<div xmlns="http://www.w3.org/1999/xhtml" class="unit-data-tables">

						<div class="table-lotv unit-expo-table active" data-expo="lotv">
								<div class="unit-datatable">
									<div class="title-bar" id="terran"><span>Terran</span><div class="terran"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														
														<th>Energy</th>
														<th>Armor</th>
													</tr>







														<tr onclick="Core.goTo('scv')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="scv"></span></span>
																</div>
																	<span>SCV</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 17</td>
															<td class="producer">
																	Command Center
															</td>
															<td class="life">45</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('marine')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="marine"></span></span>
																</div>
																	<span>Marine</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 25</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">45</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('marauder')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="marauder"></span></span>
																</div>
																	<span>Marauder</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">125</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('reaper')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="reaper"></span></span>
																</div>
																	<span>Reaper</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 45</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">60</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('ghost')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="ghost"></span></span>
																</div>
																	<span>Ghost</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 40</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">100</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('hellion')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hellion"></span></span>
																</div>
																	<span>Hellion</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">90</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('siege-tank')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="siege-tank"></span></span>
																</div>
																	<span>Siege Tank</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 125</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 45</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">160</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('thor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="thor"></span></span>
																</div>
																	<span>Thor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">400</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('viking')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="viking"></span></span>
																</div>
																	<span>Viking</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 75</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 42</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">125</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('medivac')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="medivac"></span></span>
																</div>
																	<span>Medivac</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 42</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">150</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('raven')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="raven"></span></span>
																</div>
																	<span>Raven</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">140</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('banshee')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="banshee"></span></span>
																</div>
																	<span>Banshee</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">140</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('battlecruiser')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="battlecruiser"></span></span>
																</div>
																	<span>Battlecruiser</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 400</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 300</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 90</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">550</td>
															
															<td class="energy">200</td>
															<td class="armor">3</td>
														</tr>






														<tr onclick="Core.goTo('hellbat')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hellbat"></span></span>
																</div>
																	<span>Hellbat</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">135</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('widow-mine')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="widow-mine"></span></span>
																</div>
																	<span>Widow Mine</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 75</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 40</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">90</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('liberator')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="liberator"></span></span>
																</div>
																	<span>Liberator</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">180</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('cyclone')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="cyclone"></span></span>
																</div>
																	<span>Cyclone</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 32</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">120</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
								<div class="unit-datatable">
									<div class="title-bar" id="zerg"><span>Zerg</span><div class="zerg"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														
														<th>Energy</th>
														<th>Armor</th>
													</tr>


														<tr onclick="Core.goTo('larva')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="larva"></span></span>
																</div>
																	<span>Larva</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="producer">
																	<span class="nodata">-</span>
															</td>
															<td class="life">25</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">10</td>
														</tr>






														<tr onclick="Core.goTo('drone')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="drone"></span></span>
																</div>
																	<span>Drone</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 17</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">40</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('overlord')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="overlord"></span></span>
																</div>
																	<span>Overlord</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 25</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('zergling')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="zergling"></span></span>
																</div>
																	<span>Zergling</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 24</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">35</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('queen')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="queen"></span></span>
																</div>
																	<span>Queen</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 50</td>
															<td class="producer">
																	Hatchery
															</td>
															<td class="life">175</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('hydralisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hydralisk"></span></span>
																</div>
																	<span>Hydralisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 33</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">80</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('baneling')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="baneling"></span></span>
																</div>
																	<span>Baneling</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 25</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 20</td>
															<td class="producer">
																	Zergling
															</td>
															<td class="life">30</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('overseer')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="overseer"></span></span>
																</div>
																	<span>Overseer</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 17</td>
															<td class="producer">
																	Overlord
															</td>
															<td class="life">200</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('roach')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="roach"></span></span>
																</div>
																	<span>Roach</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 75</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 27</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">145</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('infestor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="infestor"></span></span>
																</div>
																	<span>Infestor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 50</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">90</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('mutalisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mutalisk"></span></span>
																</div>
																	<span>Mutalisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 33</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">120</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('corruptor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="corruptor"></span></span>
																</div>
																	<span>Corruptor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 40</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('nydus-worm')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="nydus-worm"></span></span>
																</div>
																	<span>Nydus Worm</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 20</td>
															<td class="producer">
																	Nydus Network
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('ultralisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="ultralisk"></span></span>
																</div>
																	<span>Ultralisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 55</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">500</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('brood-lord')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="brood-lord"></span></span>
																</div>
																	<span>Brood Lord</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 34</td>
															<td class="producer">
																	Corruptor
															</td>
															<td class="life">225</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('swarm-host')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="swarm-host"></span></span>
																</div>
																	<span>Swarm Host</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 40</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">160</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('viper')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="viper"></span></span>
																</div>
																	<span>Viper</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 40</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">150</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('ravager')" class="button-rollover">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="ravager"></span></span>
																</div>
																	<span>Ravager</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 25</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 75</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 9</td>
															<td class="producer">
																	Roach
															</td>
															<td class="life">120</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('lurker')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="lurker"></span></span>
																</div>
																	<span>Lurker</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 15</td>
															<td class="producer">
																	Hydralisk
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
								<div class="unit-datatable">
									<div class="title-bar" id="protoss"><span>Protoss</span><div class="protoss"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														<th>Shields</th>
														<th>Energy</th>
														<th>Armor</th>
													</tr>







														<tr onclick="Core.goTo('probe')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="probe"></span></span>
																</div>
																	<span>Probe</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 17</td>
															<td class="producer">
																	Nexus
															</td>
															<td class="life">20</td>
															<td class="shields">20</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('zealot')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="zealot"></span></span>
																</div>
																	<span>Zealot</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 38</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">100</td>
															<td class="shields">50</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('stalker')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="stalker"></span></span>
																</div>
																	<span>Stalker</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 125</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 42</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">80</td>
															<td class="shields">80</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('sentry')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="sentry"></span></span>
																</div>
																	<span>Sentry</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 37</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">40</td>
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('observer')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="observer"></span></span>
																</div>
																	<span>Observer</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 25</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 75</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 30</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">40</td>
															<td class="shields">20</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('immortal')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="immortal"></span></span>
																</div>
																	<span>Immortal</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 250</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">200</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('warp-prism')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="warp-prism"></span></span>
																</div>
																	<span>Warp Prism</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 50</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">100</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('colossus')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="colossus"></span></span>
																</div>
																	<span>Colossus</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 75</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">200</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('phoenix')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="phoenix"></span></span>
																</div>
																	<span>Phoenix</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 35</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">120</td>
															<td class="shields">60</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('void-ray')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="void-ray"></span></span>
																</div>
																	<span>Void Ray</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 250</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 60</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">150</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('high-templar')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="high-templar"></span></span>
																</div>
																	<span>High Templar</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">40</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('dark-templar')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="dark-templar"></span></span>
																</div>
																	<span>Dark Templar</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 125</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 125</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">80</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('archon')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="archon"></span></span>
																</div>
																	<span>Archon</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 12</td>
															<td class="producer">
																	High Templar
															</td>
															<td class="life">10</td>
															<td class="shields">350</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('carrier')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="carrier"></span></span>
																</div>
																	<span>Carrier</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 350</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 250</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 120</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">250</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('mothership')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mothership"></span></span>
																</div>
																	<span>Mothership</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 300</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 100</td>
															<td class="producer">
																	Mothership Core
															</td>
															<td class="life">350</td>
															<td class="shields">350</td>
															<td class="energy">200</td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('mothership-core')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mothership-core"></span></span>
																</div>
																	<span>Mothership Core</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 30</td>
															<td class="producer">
																	Nexus
															</td>
															<td class="life">130</td>
															<td class="shields">60</td>
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('oracle')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="oracle"></span></span>
																</div>
																	<span>Oracle</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 50</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">100</td>
															<td class="shields">60</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('tempest')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="tempest"></span></span>
																</div>
																	<span>Tempest</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 60</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">300</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('adept')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="adept"></span></span>
																</div>
																	<span>Adept</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 38</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">80</td>
															<td class="shields">70</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('disruptor')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="disruptor"></span></span>
																</div>
																	<span>Disruptor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 50</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">100</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
					</div>

						<div class="table-hots unit-expo-table " data-expo="hots">
								<div class="unit-datatable">
									<div class="title-bar" id="terran"><span>Terran</span><div class="terran"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														
														<th>Energy</th>
														<th>Armor</th>
													</tr>







														<tr onclick="Core.goTo('scv')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="scv"></span></span>
																</div>
																	<span>SCV</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 17</td>
															<td class="producer">
																	Command Center
															</td>
															<td class="life">45</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('marine')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="marine"></span></span>
																</div>
																	<span>Marine</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 25</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">45</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('marauder')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="marauder"></span></span>
																</div>
																	<span>Marauder</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">125</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('reaper')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="reaper"></span></span>
																</div>
																	<span>Reaper</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 45</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">60</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('ghost')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="ghost"></span></span>
																</div>
																	<span>Ghost</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 40</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">100</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('hellion')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hellion"></span></span>
																</div>
																	<span>Hellion</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">90</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('siege-tank')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="siege-tank"></span></span>
																</div>
																	<span>Siege Tank</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 125</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 45</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">160</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('thor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="thor"></span></span>
																</div>
																	<span>Thor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">400</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('viking')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="viking"></span></span>
																</div>
																	<span>Viking</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 75</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 42</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">125</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('medivac')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="medivac"></span></span>
																</div>
																	<span>Medivac</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 42</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">150</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('raven')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="raven"></span></span>
																</div>
																	<span>Raven</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">140</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('banshee')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="banshee"></span></span>
																</div>
																	<span>Banshee</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">140</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('battlecruiser')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="battlecruiser"></span></span>
																</div>
																	<span>Battlecruiser</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 400</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 300</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 90</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">550</td>
															
															<td class="energy">200</td>
															<td class="armor">3</td>
														</tr>






														<tr onclick="Core.goTo('hellbat')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hellbat"></span></span>
																</div>
																	<span>Hellbat</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">135</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('widow-mine')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="widow-mine"></span></span>
																</div>
																	<span>Widow Mine</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 75</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 40</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">90</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
								<div class="unit-datatable">
									<div class="title-bar" id="zerg"><span>Zerg</span><div class="zerg"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														
														<th>Energy</th>
														<th>Armor</th>
													</tr>


														<tr onclick="Core.goTo('larva')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="larva"></span></span>
																</div>
																	<span>Larva</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="producer">
																	<span class="nodata">-</span>
															</td>
															<td class="life">25</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">10</td>
														</tr>






														<tr onclick="Core.goTo('drone')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="drone"></span></span>
																</div>
																	<span>Drone</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 17</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">40</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('overlord')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="overlord"></span></span>
																</div>
																	<span>Overlord</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 25</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('zergling')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="zergling"></span></span>
																</div>
																	<span>Zergling</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 24</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">35</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('queen')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="queen"></span></span>
																</div>
																	<span>Queen</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 50</td>
															<td class="producer">
																	Hatchery
															</td>
															<td class="life">175</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('hydralisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hydralisk"></span></span>
																</div>
																	<span>Hydralisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 33</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">80</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('baneling')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="baneling"></span></span>
																</div>
																	<span>Baneling</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 25</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 20</td>
															<td class="producer">
																	Zergling
															</td>
															<td class="life">30</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('overseer')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="overseer"></span></span>
																</div>
																	<span>Overseer</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 17</td>
															<td class="producer">
																	Overlord
															</td>
															<td class="life">200</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('roach')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="roach"></span></span>
																</div>
																	<span>Roach</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 75</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 27</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">145</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('infestor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="infestor"></span></span>
																</div>
																	<span>Infestor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 50</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">90</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('mutalisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mutalisk"></span></span>
																</div>
																	<span>Mutalisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 33</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">120</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('corruptor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="corruptor"></span></span>
																</div>
																	<span>Corruptor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 40</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('nydus-worm')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="nydus-worm"></span></span>
																</div>
																	<span>Nydus Worm</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 20</td>
															<td class="producer">
																	Nydus Network
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('ultralisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="ultralisk"></span></span>
																</div>
																	<span>Ultralisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 55</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">500</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('brood-lord')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="brood-lord"></span></span>
																</div>
																	<span>Brood Lord</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 34</td>
															<td class="producer">
																	Corruptor
															</td>
															<td class="life">225</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('swarm-host')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="swarm-host"></span></span>
																</div>
																	<span>Swarm Host</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 40</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">160</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('viper')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="viper"></span></span>
																</div>
																	<span>Viper</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 40</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">150</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
								<div class="unit-datatable">
									<div class="title-bar" id="protoss"><span>Protoss</span><div class="protoss"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														<th>Shields</th>
														<th>Energy</th>
														<th>Armor</th>
													</tr>







														<tr onclick="Core.goTo('probe')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="probe"></span></span>
																</div>
																	<span>Probe</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 17</td>
															<td class="producer">
																	Nexus
															</td>
															<td class="life">20</td>
															<td class="shields">20</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('zealot')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="zealot"></span></span>
																</div>
																	<span>Zealot</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 38</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">100</td>
															<td class="shields">50</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('stalker')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="stalker"></span></span>
																</div>
																	<span>Stalker</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 125</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 42</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">80</td>
															<td class="shields">80</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('sentry')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="sentry"></span></span>
																</div>
																	<span>Sentry</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 37</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">40</td>
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('observer')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="observer"></span></span>
																</div>
																	<span>Observer</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 25</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 75</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 30</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">40</td>
															<td class="shields">20</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('immortal')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="immortal"></span></span>
																</div>
																	<span>Immortal</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 250</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">200</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('warp-prism')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="warp-prism"></span></span>
																</div>
																	<span>Warp Prism</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 50</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">100</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('colossus')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="colossus"></span></span>
																</div>
																	<span>Colossus</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 75</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">200</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('phoenix')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="phoenix"></span></span>
																</div>
																	<span>Phoenix</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 35</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">120</td>
															<td class="shields">60</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('void-ray')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="void-ray"></span></span>
																</div>
																	<span>Void Ray</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 250</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 60</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">150</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('high-templar')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="high-templar"></span></span>
																</div>
																	<span>High Templar</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">40</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('dark-templar')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="dark-templar"></span></span>
																</div>
																	<span>Dark Templar</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 125</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 125</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">80</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('archon')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="archon"></span></span>
																</div>
																	<span>Archon</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 12</td>
															<td class="producer">
																	High Templar
															</td>
															<td class="life">10</td>
															<td class="shields">350</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('carrier')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="carrier"></span></span>
																</div>
																	<span>Carrier</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 350</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 250</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 120</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">300</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('mothership')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mothership"></span></span>
																</div>
																	<span>Mothership</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 300</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 100</td>
															<td class="producer">
																	Mothership Core
															</td>
															<td class="life">350</td>
															<td class="shields">350</td>
															<td class="energy">200</td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('mothership-core')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mothership-core"></span></span>
																</div>
																	<span>Mothership Core</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 30</td>
															<td class="producer">
																	Nexus
															</td>
															<td class="life">130</td>
															<td class="shields">60</td>
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('oracle')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="oracle"></span></span>
																</div>
																	<span>Oracle</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 50</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">100</td>
															<td class="shields">60</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('tempest')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="tempest"></span></span>
																</div>
																	<span>Tempest</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 60</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">300</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
					</div>

						<div class="table-wol unit-expo-table " data-expo="wol">
								<div class="unit-datatable">
									<div class="title-bar" id="terran"><span>Terran</span><div class="terran"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														
														<th>Energy</th>
														<th>Armor</th>
													</tr>







														<tr onclick="Core.goTo('scv')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="scv"></span></span>
																</div>
																	<span>SCV</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 17</td>
															<td class="producer">
																	Command Center
															</td>
															<td class="life">45</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('marine')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="marine"></span></span>
																</div>
																	<span>Marine</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 25</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">45</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('marauder')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="marauder"></span></span>
																</div>
																	<span>Marauder</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">125</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('reaper')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="reaper"></span></span>
																</div>
																	<span>Reaper</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 45</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">50</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('ghost')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="ghost"></span></span>
																</div>
																	<span>Ghost</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 40</td>
															<td class="producer">
																	Barracks
															</td>
															<td class="life">100</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('hellion')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hellion"></span></span>
																</div>
																	<span>Hellion</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 30</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">90</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('siege-tank')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="siege-tank"></span></span>
																</div>
																	<span>Siege Tank</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 125</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 45</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">160</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('thor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="thor"></span></span>
																</div>
																	<span>Thor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Factory
															</td>
															<td class="life">400</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('viking')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="viking"></span></span>
																</div>
																	<span>Viking</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 75</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 42</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">125</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('medivac')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="medivac"></span></span>
																</div>
																	<span>Medivac</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 42</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">150</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('raven')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="raven"></span></span>
																</div>
																	<span>Raven</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">140</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('banshee')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="banshee"></span></span>
																</div>
																	<span>Banshee</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 60</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">140</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('battlecruiser')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="battlecruiser"></span></span>
																</div>
																	<span>Battlecruiser</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 400</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-terran.gif" alt="" /> 300</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-terran.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-terran.gif" alt="" /> 90</td>
															<td class="producer">
																	Starport
															</td>
															<td class="life">550</td>
															
															<td class="energy">200</td>
															<td class="armor">3</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
								<div class="unit-datatable">
									<div class="title-bar" id="zerg"><span>Zerg</span><div class="zerg"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														
														<th>Energy</th>
														<th>Armor</th>
													</tr>


														<tr onclick="Core.goTo('larva')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="larva"></span></span>
																</div>
																	<span>Larva</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="producer">
																	<span class="nodata">-</span>
															</td>
															<td class="life">25</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">10</td>
														</tr>






														<tr onclick="Core.goTo('drone')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="drone"></span></span>
																</div>
																	<span>Drone</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 17</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">40</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('overlord')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="overlord"></span></span>
																</div>
																	<span>Overlord</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 25</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('zergling')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="zergling"></span></span>
																</div>
																	<span>Zergling</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 24</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">35</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('queen')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="queen"></span></span>
																</div>
																	<span>Queen</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 50</td>
															<td class="producer">
																	Hatchery
															</td>
															<td class="life">175</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('hydralisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="hydralisk"></span></span>
																</div>
																	<span>Hydralisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 33</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">80</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('baneling')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="baneling"></span></span>
																</div>
																	<span>Baneling</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 25</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 20</td>
															<td class="producer">
																	Zergling
															</td>
															<td class="life">30</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('overseer')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="overseer"></span></span>
																</div>
																	<span>Overseer</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 17</td>
															<td class="producer">
																	Overlord
															</td>
															<td class="life">200</td>
															
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('roach')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="roach"></span></span>
																</div>
																	<span>Roach</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 75</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 25</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 27</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">145</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('infestor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="infestor"></span></span>
																</div>
																	<span>Infestor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 50</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">90</td>
															
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('mutalisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mutalisk"></span></span>
																</div>
																	<span>Mutalisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 33</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">120</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('corruptor')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="corruptor"></span></span>
																</div>
																	<span>Corruptor</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 40</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('nydus-worm')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="nydus-worm"></span></span>
																</div>
																	<span>Nydus Worm</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 20</td>
															<td class="producer">
																	Nydus Network
															</td>
															<td class="life">200</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('ultralisk')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="ultralisk"></span></span>
																</div>
																	<span>Ultralisk</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 55</td>
															<td class="producer">
																	Larva
															</td>
															<td class="life">500</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('brood-lord')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="brood-lord"></span></span>
																</div>
																	<span>Brood Lord</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-zerg.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-zerg.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-zerg.gif" alt="" /> 34</td>
															<td class="producer">
																	Corruptor
															</td>
															<td class="life">225</td>
															
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
								<div class="unit-datatable">
									<div class="title-bar" id="protoss"><span>Protoss</span><div class="protoss"></div></div>
									<div class="databox">
										<div class="wapper1">
											<div class="wapper2">
												<table>
													<tr class="titles">
														<th class="title-unit">Unit</th>
														<th>Mineral</th>
														<th>Vespene</th>
														<th>Supply</th>
														<th>Build Time</th>
														<th>Build From</th>
														<th>Life</th>
														<th>Shields</th>
														<th>Energy</th>
														<th>Armor</th>
													</tr>







														<tr onclick="Core.goTo('probe')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="probe"></span></span>
																</div>
																	<span>Probe</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 17</td>
															<td class="producer">
																	Nexus
															</td>
															<td class="life">20</td>
															<td class="shields">20</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('zealot')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="zealot"></span></span>
																</div>
																	<span>Zealot</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 100</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 38</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">100</td>
															<td class="shields">50</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('stalker')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="stalker"></span></span>
																</div>
																	<span>Stalker</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 125</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 50</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 42</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">80</td>
															<td class="shields">80</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('sentry')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="sentry"></span></span>
																</div>
																	<span>Sentry</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 37</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">40</td>
															<td class="energy">200</td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('observer')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="observer"></span></span>
																</div>
																	<span>Observer</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 25</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 75</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 1</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 30</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">40</td>
															<td class="shields">20</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('immortal')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="immortal"></span></span>
																</div>
																	<span>Immortal</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 250</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">200</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('warp-prism')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="warp-prism"></span></span>
																</div>
																	<span>Warp Prism</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 200</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 50</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">100</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('colossus')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="colossus"></span></span>
																</div>
																	<span>Colossus</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 300</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 200</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 75</td>
															<td class="producer">
																	Robotics Facility
															</td>
															<td class="life">200</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('phoenix')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="phoenix"></span></span>
																</div>
																	<span>Phoenix</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 150</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 100</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 35</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">120</td>
															<td class="shields">60</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('void-ray')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="void-ray"></span></span>
																</div>
																	<span>Void Ray</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 250</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 3</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 60</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">150</td>
															<td class="shields">100</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('high-templar')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="high-templar"></span></span>
																</div>
																	<span>High Templar</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 50</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 150</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">40</td>
															<td class="energy">200</td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('dark-templar')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="dark-templar"></span></span>
																</div>
																	<span>Dark Templar</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 125</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 125</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 2</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 55</td>
															<td class="producer">
																	Gateway
															</td>
															<td class="life">40</td>
															<td class="shields">80</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">1</td>
														</tr>






														<tr onclick="Core.goTo('archon')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="archon"></span></span>
																</div>
																	<span>Archon</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> <span class="nodata">-</span></td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 4</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 12</td>
															<td class="producer">
																	High Templar
															</td>
															<td class="life">10</td>
															<td class="shields">350</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">0</td>
														</tr>






														<tr onclick="Core.goTo('carrier')" class="button-rollover ">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="carrier"></span></span>
																</div>
																	<span>Carrier</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 350</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 250</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 6</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 120</td>
															<td class="producer">
																	Stargate
															</td>
															<td class="life">300</td>
															<td class="shields">150</td>
															<td class="energy"><span class="nodata">-</span></td>
															<td class="armor">2</td>
														</tr>






														<tr onclick="Core.goTo('mothership')" class="button-rollover last">
															<td class="unit-title">
																<div class="portrait-c">
																	<span class="unit-thumb-54x49"><span class="mothership"></span></span>
																</div>
																	<span>Mothership</span>
															</td>
															<td class="mineral"><img src="/sc2/static/images/game/unit/icon-mineral.gif" alt="" /> 400</td>
															<td class="vespene"><img src="/sc2/static/images/game/unit/icon-vespene-protoss.gif" alt="" /> 400</td>
															<td class="supply"><img src="/sc2/static/images/game/unit/icon-supply-protoss.gif" alt="" /> 8</td>
															<td class="buildtime"><img src="/sc2/static/images/game/unit/icon-buildtime-protoss.gif" alt="" /> 160</td>
															<td class="producer">
																	Nexus
															</td>
															<td class="life">350</td>
															<td class="shields">350</td>
															<td class="energy">200</td>
															<td class="armor">2</td>
														</tr>

												</table>
												<div class="p-top-left"></div>
												<div class="p-bottom-right"></div>
											</div>
										</div>
									</div>
								</div>
					</div>
			</div>